In [1]:
# closest location Cambridge Maryland
# https://tidesandcurrents.noaa.gov/waterlevels.html?id=8571892

# Link for CSV:
# https://api.tidesandcurrents.noaa.gov/api/prod/datagetter
#  ?product=water_level&application=NOS.COOPS.TAC.WL
#  &begin_date=20250201&end_date=20250202&datum=MLLW
#  &station=8571892&time_zone=GMT&units=english&format=csv

import pandas as pd
import sqlite3
import csv
import requests

In [2]:
# Link for hourly CSV:
# https://api.tidesandcurrents.noaa.gov/api/prod/datagetter
#  ?product=hourly_height&application=NOS.COOPS.TAC.WL
#  &begin_date=19790601&end_date=19790701&datum=MLLW
#  &station=8571892&time_zone=GMT&units=english&format=CSV


In [3]:
conn = sqlite3.connect('water_levels.db')
scur = conn.cursor()

In [4]:
table_name = 'noaa_data'


In [5]:
scur = conn.cursor()
scur.execute(f'''
select Station, min(Date_Time), max(Date_Time), 
count(distinct(date(Date_Time,'start of year'))) as years_reported,
count(distinct(date(Date_Time,'start of month'))) as months_reported,
count(distinct(date(Date_Time,'start of day'))) as days_reported,
count(*) as total_records
from {table_name}
group by 1
order by 1''')

for col in scur.description:
    print(col[0])

for row in scur.fetchall():
    print(row)

Station
min(Date_Time)
max(Date_Time)
years_reported
months_reported
days_reported
total_records
('8571892', '1979-06-01 01:00', '2024-12-31 23:54', 46, 544, 16520, 2576856)
('8594900', '1996-01-01 00:00', '2024-12-31 23:54', 29, 338, 10261, 2594305)


In [6]:
station = '8571892' ## Cambridge, MD
station = '8575512' ## Annapolis, MD

In [7]:
scur = conn.cursor()
scur.execute(f'''
select Station, 
date(Date_Time,'start of year'),
min(Date_Time),
count(*) as total_records
from {table_name}
where station = {station}
group by 1,2
order by 1,2 desc
''')

for col in scur.description:
    print(col[0])

for row in scur.fetchall():
    print(row)

Station
date(Date_Time,'start of year')
min(Date_Time)
total_records


In [8]:
### URL Builder based on month, year, station
def url(year,month,station):
    first_day_of_month = 1
    short_mths = [9,4,6,11]
    if month == 2:  # February
        if year % 4 == 0:  #leap year
            last_day_of_month = 29
        else:
            last_day_of_month = 28
    elif month in short_mths:
        last_day_of_month = 30
    else:
        last_day_of_month = 31

    url = f"""https://api.tidesandcurrents.noaa.gov/api/prod/datagetter
?product=water_level&application=NOS.COOPS.TAC.WL
&begin_date={year}{month:02d}{first_day_of_month:02d}&end_date={year}{month:02d}{last_day_of_month}&datum=MLLW
&station={station}&time_zone=GMT&units=english&format=csv"""

    return (url.replace('\n',''))

year = 2000
month = 2
station = '8571892'
print(url(year,month,station))

https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=20000201&end_date=20000229&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv


In [9]:
### URL Builder based on month, year, station
### This does for the hourly height, as opposed to the water level
def alt_url(year,month,station):
    first_day_of_month = 1
    short_mths = [9,4,6,11]
    if month == 2:  # February
        if year % 4 == 0:  #leap year
            last_day_of_month = 29
        else:
            last_day_of_month = 28
    elif month in short_mths:
        last_day_of_month = 30
    else:
        last_day_of_month = 31

    url = f"""https://api.tidesandcurrents.noaa.gov/api/prod/datagetter
?product=hourly_height&application=NOS.COOPS.TAC.WL
&begin_date={year}{month:02d}{first_day_of_month:02d}&end_date={year}{month:02d}{last_day_of_month}&datum=MLLW
&station={station}&time_zone=GMT&units=english&format=csv"""

    return (url.replace('\n',''))

year = 2000
month = 2
station = '8571892'
print(alt_url(year,month,station))

https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=hourly_height&application=NOS.COOPS.TAC.WL&begin_date=20000201&end_date=20000229&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv


In [10]:
# check some dates:
year = 1997
station = 8571892
for month in range(5,3,-1):
    print(month,url(year,month,station))
    print(month,alt_url(year,month,station))


5 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970501&end_date=19970531&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
5 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=hourly_height&application=NOS.COOPS.TAC.WL&begin_date=19970501&end_date=19970531&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
4 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=water_level&application=NOS.COOPS.TAC.WL&begin_date=19970401&end_date=19970430&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv
4 https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=hourly_height&application=NOS.COOPS.TAC.WL&begin_date=19970401&end_date=19970430&datum=MLLW&station=8571892&time_zone=GMT&units=english&format=csv


In [12]:
### GET YEARS WORTH OF DATA
def year_upload(station, year, table_name):
    print(year)
    record_count = 0
    for month in range(12,0,-1):
        record_count += month_upload(station, year, month, table_name)

    conn.commit()
    return record_count

### GET MONTH OF DATA
def month_upload(station, year, month, table_name):
    print(year, month)
    #print(month,url(year,month,station))
    response = requests.get(url(year,month,station))
    alt_url_flag = False
    if "No data was found" in response.text:
        response = requests.get(alt_url(year,month,station))
        alt_url_flag = True
        if "No data was found" in response.text:
            print(f"No data for {year}, {month}")
            return 0

    #print(len(response.text))
    reader = csv.reader(response.text.splitlines())
    columns = next(reader)
    record_count = 0

    for line in reader:
        #print(line)
        try:
            if alt_url_flag: ## the alt url misses a few values
                placeholders = ", ".join([f"'{val}'" for val in line]) + ", '0', '0', 'v', " + str(station)
            else:
                placeholders = ", ".join([f"'{val}'" for val in line]) + ", " + str(station)
            record_count += 1
            query = f'INSERT INTO {table_name} VALUES ({placeholders})'
            conn.execute(query)
        except Exception as error_msg:
            print(f"Error {error_msg}")
    conn.commit()
    
    return record_count

In [13]:
station = '8575512' ## Annapolis
for year in range(2024,1927,-1):
    print(f"{year} has {year_upload(station, year, table_name)} records")


2024
2024 12
2024 11
2024 10
2024 9
2024 8
2024 7
2024 6
2024 5
2024 4
2024 3
2024 2
2024 1
2024 has 87840 records
2023
2023 12
2023 11
2023 10
2023 9
2023 8
2023 7
2023 6
2023 5
2023 4
2023 3
2023 2
2023 1
2023 has 87600 records
2022
2022 12
2022 11
2022 10
2022 9
2022 8
2022 7
2022 6
2022 5
2022 4
2022 3
2022 2
2022 1
2022 has 87600 records
2021
2021 12
2021 11
2021 10
2021 9
2021 8
2021 7
2021 6
2021 5
2021 4
2021 3
2021 2
2021 1
2021 has 87600 records
2020
2020 12
2020 11
2020 10
2020 9
2020 8
2020 7
2020 6
2020 5
2020 4
2020 3
2020 2
2020 1
2020 has 87840 records
2019
2019 12
2019 11
2019 10
2019 9
2019 8
2019 7
2019 6
2019 5
2019 4
2019 3
2019 2
2019 1
2019 has 87600 records
2018
2018 12
2018 11
2018 10
2018 9
2018 8
2018 7
2018 6
2018 5
2018 4
2018 3
2018 2
2018 1
2018 has 87600 records
2017
2017 12
2017 11
2017 10
2017 9
2017 8
2017 7
2017 6
2017 5
2017 4
2017 3
2017 2
2017 1
2017 has 87600 records
2016
2016 12
2016 11
2016 10
2016 9
2016 8
2016 7
2016 6
2016 5
2016 4
2016 3
20

1955 5
1955 4
1955 3
1955 2
1955 1
1955 has 8760 records
1954
1954 12
1954 11
1954 10
1954 9
1954 8
1954 7
1954 6
1954 5
1954 4
1954 3
1954 2
1954 1
1954 has 8760 records
1953
1953 12
1953 11
1953 10
1953 9
1953 8
1953 7
1953 6
1953 5
1953 4
1953 3
1953 2
1953 1
1953 has 8760 records
1952
1952 12
1952 11
1952 10
1952 9
1952 8
1952 7
1952 6
1952 5
1952 4
1952 3
1952 2
1952 1
1952 has 8784 records
1951
1951 12
1951 11
1951 10
1951 9
1951 8
1951 7
1951 6
1951 5
1951 4
1951 3
1951 2
1951 1
1951 has 8760 records
1950
1950 12
1950 11
1950 10
1950 9
1950 8
1950 7
1950 6
1950 5
1950 4
1950 3
1950 2
1950 1
1950 has 8760 records
1949
1949 12
1949 11
1949 10
1949 9
1949 8
1949 7
1949 6
1949 5
1949 4
1949 3
1949 2
1949 1
1949 has 8760 records
1948
1948 12
1948 11
1948 10
1948 9
1948 8
1948 7
1948 6
1948 5
1948 4
1948 3
1948 2
1948 1
1948 has 8784 records
1947
1947 12
1947 11
1947 10
1947 9
1947 8
1947 7
1947 6
1947 5
1947 4
1947 3
1947 2
1947 1
1947 has 8760 records
1946
1946 12
1946 11
1946 10
19

In [ ]:
station = '8594900' ### Washington DC
for year in range(1996,1995,-1):
    year_upload(station, year)


In [ ]:
month = 12
year = 2003
station = '8594900'
print(url(year,month,station))
response = requests.get(url(year,month,station))

print(response)

reader = csv.reader(response.text.splitlines())
columns = next(reader)

# create table with columns, already done

for line in reader:
    placeholders = ", ".join([f"'{val}'" for val in line]) + ", " + station
print(placeholders)